In [ ]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install -U sentence-transformers
!pip install datasets
!pip install unidecode
!wget https://raw.githubusercontent.com/CSDL-UMD/google-relation-extraction-corpus-augmented/master/dob_augment-200526.json
!wget https://raw.githubusercontent.com/CSDL-UMD/google-relation-extraction-corpus-augmented/master/pob_augment-200526.json
!wget https://raw.githubusercontent.com/CSDL-UMD/google-relation-extraction-corpus-augmented/master/pod_augment-200526.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f9858db24073c9c103277c34650c738ba8f26e25a571d6448b3991df8565068e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built senten

In [ ]:
!wget -c 'https://s3.amazonaws.com/models.huggingface.co/bert/transfo-xl-wt103-vocab.bin' -O 'vocab.bin'
!python -c 'import torch; print(*torch.load("vocab.bin")["sym2idx"].keys(), sep="\n")' | sort > vocab.txt
with open("vocab.txt", "r") as f:
  lines = f.readlines()
  vocab = [x.strip() for x in lines]

--2023-11-13 11:37:32--  https://s3.amazonaws.com/models.huggingface.co/bert/transfo-xl-wt103-vocab.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.44.6, 16.182.109.152, 54.231.135.160, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.44.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9143613 (8.7M) [application/octet-stream]
Saving to: ‘vocab.bin’

vocab.bin           100%[===================>]   8.72M  31.4MB/s    in 0.3s    

2023-11-13 11:37:32 (31.4 MB/s) - ‘vocab.bin’ saved [9143613/9143613]



In [ ]:
import pandas as pd
from transformers import AutoModelForMaskedLM, AutoConfig
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
import torch
import collections
import numpy as np
from transformers import default_data_collator
from datasets import Dataset
from huggingface_hub import login
from transformers import TrainingArguments
from transformers import Trainer
import math
from transformers import pipeline
from transformers import DistilBertTokenizerFast
from sklearn.model_selection import train_test_split
from datetime import datetime
import re
from transformers import DistilBertForQuestionAnswering
from torch.utils.data import DataLoader
from transformers import AdamW
from sentence_transformers import SentenceTransformer, util
from datetime import datetime
import random as rd
import unidecode

In [ ]:
model_checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### ***Data pre-processing according to some rules used in paper***

In [ ]:
df = pd.read_json('/content/dob_augment-200526.json')
df = df.where(df['maj_vote']=='yes')
df = df.dropna(axis=0, how='all').reset_index()
for i in range(len(df.evidences)):
  df.evidences[i] = df.evidences[i][0].get('snippet')
df = df[['pred', 'sub', 'obj', 'evidences']]
df = df.rename(columns={"sub": "subj"}, errors="raise")
df = df.apply(lambda x: x.astype(str).str.lower())
subj_new = []
obj_new = []
evidence_new = []
for subj, obj, evid in zip(df['subj'], df['obj'], df['evidences']):
  if obj.startswith('0') and obj.isnumeric():
    obj = str(int(obj))
  if obj not in evid:
    date_obj = datetime.strptime(obj, '%Y-%m-%d')
    fmt_date = fr"{str(date_obj.day)}.*{date_obj.strftime('%B').lower()}.*{str(date_obj.year)}"
    fmt_date_2 = fr"{date_obj.strftime('%B').lower()}.*{str(date_obj.day)}.*{str(date_obj.year)}"
    if re.search(fmt_date, evid):
      evid = re.sub(fmt_date, obj.split("-")[0], evid)
      subj_new.append(subj)
      obj_new.append(obj.split("-")[0])
      evidence_new.append(evid)
    elif re.search(fmt_date_2, evid):
      evid = re.sub(fmt_date_2, obj.split("-")[0], evid)
      subj_new.append(subj)
      obj_new.append(obj.split("-")[0])
      evidence_new.append(evid)
  else:
    subj_new.append(subj)
    obj_new.append(obj)
    evidence_new.append(evid)

pred_new = ['/people/person/date_of_birth'] * len(subj_new)
df = pd.DataFrame(list(zip(pred_new, subj_new, obj_new, evidence_new)), columns=df.columns.to_list())
dataset = Dataset.from_pandas(df)

<ipython-input-5-4fd7e33f1d53>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.evidences[i] = df.evidences[i][0].get('snippet')


In [ ]:
df_2 = pd.read_json('/content/pob_augment-200526.json')
df_2 = df_2.where(df_2['maj_vote']=='yes')
df_2 = df_2.dropna(axis=0, how='all').reset_index()
for i in range(len(df_2.evidences)):
  df_2.evidences[i] = df_2.evidences[i][0].get('snippet')
df_2 = df_2[['pred', 'sub', 'obj', 'evidences']]
subj_new = []
obj_new = []
evidence_new = []
for subj, evid, obj in zip(df_2['sub'], df_2['evidences'], df_2['obj']):
  if obj in evid and len(tokenizer(evid)['input_ids']) < 100 and obj in vocab:
    subj_new.append(subj)
    obj_new.append(obj)
    evidence_new.append(evid)
pred_new = ['/people/person/place_of_birth'] * len(subj_new)
df_2 = pd.DataFrame(list(zip(pred_new, subj_new, obj_new, evidence_new)), columns=df_2.columns.to_list())
df_2 = df_2.rename(columns={"sub": "subj"}, errors="raise")
df_2 = df_2.apply(lambda x: x.astype(str).str.lower())
df_2.obj = df_2.obj.apply(lambda x: unidecode.unidecode(x))
subj_new = []
obj_new = []
evidence_new = []
for subj,obj,evid in zip(df_2.subj,df_2.obj, df_2.evidences):
  if obj in evid:
    subj_new.append(subj)
    obj_new.append(obj)
    evidence_new.append(evid)

pred_new = ['/people/person/place_of_birth'] * len(subj_new)
df_2 = pd.DataFrame(list(zip(pred_new, subj_new, obj_new, evidence_new)), columns=df_2.columns.to_list())
dataset_2 = Dataset.from_pandas(df_2)

<ipython-input-6-be5cee283402>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.evidences[i] = df_2.evidences[i][0].get('snippet')
Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
df_conc = pd.concat([df,df_2], ignore_index=True)

In [ ]:
dataset = Dataset.from_pandas(df_conc)

In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model=model, tokenizer=tokenizer
)

### ***Non-optimal queries for date of birth***


*   born [MASK]
*   born c. [MASK]
*   born ca. [MASK]
*   (born c. [MASK])
*   (born ca. [MASK])
*   born in year [MASK]



In [ ]:
count_pred_dob = 0
for (sub, pred) in zip(df['subj'], df['pred']):
  if pred == '/people/person/date_of_birth':
    text = sub + " born [MASK]."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  if df[(df['subj']== sub)].obj.values[0] == preds[0]['token_str']:
    count_pred_dob += 1
print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 0, total cases number: 2034, precision: 0.0


In [ ]:
count_pred_dob = 0
for (sub, pred) in zip(df['subj'], df['pred']):
  if pred == '/people/person/date_of_birth':
    text = sub + " born c. [MASK]."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  num_tok = [preds[i]['token_str'] for i in range(len(preds)) if preds[i]['token_str'].isnumeric()]
  if len(num_tok) != 0:
    if df[(df['subj']== sub)].obj.values[0] == num_tok[0]:
      count_pred_dob += 1
print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 5, total cases number: 2034, precision: 0.0024582104228121925


In [ ]:
count_pred_dob = 0
for (sub, pred) in zip(df['subj'], df['pred']):
  if pred == '/people/person/date_of_birth':
    text = sub + " born ca. [MASK]."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  num_tok = [preds[i]['token_str'] for i in range(len(preds)) if preds[i]['token_str'].isnumeric()]
  if len(num_tok) != 0:
    if df[(df['subj']== sub)].obj.values[0] == num_tok[0]:
      count_pred_dob += 1
print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 0, total cases number: 2034, precision: 0.0


In [ ]:
count_pred_dob = 0
for (sub, pred) in zip(df['subj'], df['pred']):
  if pred == '/people/person/date_of_birth':
    text = sub + " (born c. [MASK])."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  num_tok = [preds[i]['token_str'] for i in range(len(preds)) if preds[i]['token_str'].isnumeric()]
  if len(num_tok) != 0:
    if df[(df['subj']== sub)].obj.values[0] == num_tok[0]:
      count_pred_dob += 1
print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 22, total cases number: 2034, precision: 0.010816125860373648


In [ ]:
count_pred_dob = 0
for (sub, pred) in zip(df['subj'], df['pred']):
  if pred == '/people/person/date_of_birth':
    text = sub + " (born ca. [MASK])."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  num_tok = [preds[i]['token_str'] for i in range(len(preds)) if preds[i]['token_str'].isnumeric()]
  if len(num_tok) != 0:
    if df[(df['subj']== sub)].obj.values[0] == num_tok[0]:
      count_pred_dob += 1
print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 24, total cases number: 2034, precision: 0.011799410029498525


In [ ]:
count_pred_dob = 0
for (sub, pred) in zip(df['subj'], df['pred']):
  if pred == '/people/person/date_of_birth':
    text = sub + " born in year [MASK]."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  num_tok = [preds[i]['token_str'] for i in range(len(preds)) if preds[i]['token_str'].isnumeric()]
  if len(num_tok) != 0:
    if df[(df['subj']== sub)].obj.values[0] == num_tok[0]:
      count_pred_dob += 1
print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 0, total cases number: 2034, precision: 0.0


### ***Optimal query***

In [ ]:
count_pred_dob = 0
for (sub, pred) in zip(df['subj'], df['pred']):
  if pred == '/people/person/date_of_birth':
    text = sub + " (born [MASK])."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  num_tok = [preds[i]['token_str'] for i in range(len(preds)) if preds[i]['token_str'].isnumeric()]
  if len(num_tok) != 0:
    if df[(df['subj']== sub)].obj.values[0] == num_tok[0]:
      count_pred_dob += 1
print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 38, total cases number: 2034, precision: 0.018682399213372666


### ***Adding context***

In [ ]:
count_pred_dob = 0
for (sub, pred) in zip(df['subj'], df['pred']):
  if pred == '/people/person/date_of_birth':
    text = "steve jobs (born 1955). " + sub + " (born [MASK])."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  num_tok = [preds[i]['token_str'] for i in range(len(preds)) if preds[i]['token_str'].isnumeric()]
  if len(num_tok) != 0:
    if df[(df['subj']== sub)].obj.values[0] == num_tok[0]:
      count_pred_dob += 1
print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 25, total cases number: 2034, precision: 0.012291052114060964


In [ ]:
count_pred_dob = 0
for (sub, pred) in zip(df['subj'], df['pred']):
  if pred == '/people/person/date_of_birth':
    text = "albert einstein (born 1879). steve jobs (born 1955). " + sub + " (born [MASK])."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  num_tok = [preds[i]['token_str'] for i in range(len(preds)) if preds[i]['token_str'].isnumeric()]
  if len(num_tok) != 0:
    if df[(df['subj']== sub)].obj.values[0] == num_tok[0]:
      count_pred_dob += 1
print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 24, total cases number: 2034, precision: 0.011799410029498525


In [ ]:
count_pred_dob = 0
for (sub, pred) in zip(df['subj'], df['pred']):
  if pred == '/people/person/date_of_birth':
    text = "friedrich hegel (born 1770). albert einstein (born 1879). steve jobs (born 1955). " + sub + " (born [MASK])."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  num_tok = [preds[i]['token_str'] for i in range(len(preds)) if preds[i]['token_str'].isnumeric()]
  if len(num_tok) != 0:
    if df[(df['subj']== sub)].obj.values[0] == num_tok[0]:
      count_pred_dob += 1
print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 23, total cases number: 2034, precision: 0.011307767944936086


### ***Place of birth***

In [ ]:
count_pred_pob = 0
for (sub, pred) in zip(df_2['subj'], df_2['pred']):
  if pred == '/people/person/place_of_birth':
    text = sub + " was born in [MASK]."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  if df_2[(df_2['subj']== sub)].obj.values[0] == preds[0]['token_str']:
    count_pred_pob += 1
print(f'count: {count_pred_pob}, total cases number: {len(df_2)}, precision: {count_pred_pob/len(df_2)}')

count: 352, total cases number: 3418, precision: 0.10298420128730251


### ***Adding context***

In [ ]:
count_pred_pob = 0
for (sub, pred) in zip(df_2['subj'], df_2['pred']):
  if pred == '/people/person/place_of_birth':
    text = "friedrich hegel was born in stuttgart. bill gates was born in seattle. " + sub + " was born in [MASK]."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  if df_2[(df_2['subj']== sub)].obj.values[0] == preds[0]['token_str']:
    count_pred_pob += 1
print(f'count: {count_pred_pob}, total cases number: {len(df_2)}, precision: {count_pred_pob/len(df_2)}')

count: 194, total cases number: 3418, precision: 0.05675833820947923


In [ ]:
count_pred_pob = 0
for (sub, pred) in zip(df_2['subj'], df_2['pred']):
  if pred == '/people/person/place_of_birth':
    text = "luciano pavarotti was born in modena. friedrich hegel was born in stuttgart. bill gates was born in seattle. " + sub + " was born in [MASK]."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  if df_2[(df_2['subj']== sub)].obj.values[0] == preds[0]['token_str']:
    count_pred_pob += 1
print(f'count: {count_pred_pob}, total cases number: {len(df_2)}, precision: {count_pred_pob/len(df_2)}')

count: 265, total cases number: 3418, precision: 0.077530719719134


### ***P@k for non-optimal and optimal queries for both DOB and POB***

In [ ]:
k= 5
for test_k in range(k):
  count_pred_dob = 0
  for (sub, pred) in zip(df['subj'], df['pred']):
    if pred == '/people/person/date_of_birth':
      text = sub + " born c. [MASK]."
    else:
      print(f'{pred} is not a canonical case')
      break
    preds = mask_filler(text)
    num_tok = [preds[i]['token_str'] for i in range(len(preds)) if preds[i]['token_str'].isnumeric()]
    if len(num_tok) != 0:
      preds_arr = []
      if len(num_tok) > test_k+1:
        preds_arr = num_tok[:test_k+1]
      if len(num_tok) <= test_k+1:
        preds_arr = num_tok
      if df[(df['subj']== sub)].obj.values[0] in preds_arr:
        count_pred_dob += 1
  print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 5, total cases number: 2464, precision: 0.002029220779220779
count: 9, total cases number: 2464, precision: 0.0036525974025974025
count: 14, total cases number: 2464, precision: 0.005681818181818182
count: 17, total cases number: 2464, precision: 0.006899350649350649
count: 17, total cases number: 2464, precision: 0.006899350649350649


In [ ]:
k= 5
for test_k in range(k):
  count_pred_dob = 0
  for (sub, pred) in zip(df['subj'], df['pred']):
    if pred == '/people/person/date_of_birth':
      text = sub + " (born [MASK])."
    else:
      print(f'{pred} is not a canonical case')
      break
    preds = mask_filler(text)
    num_tok = [preds[i]['token_str'] for i in range(len(preds)) if preds[i]['token_str'].isnumeric()]
    if len(num_tok) != 0:
      preds_arr = []
      if len(num_tok) > test_k+1:
        preds_arr = num_tok[:test_k+1]
      if len(num_tok) <= test_k+1:
        preds_arr = num_tok
      if df[(df['subj']== sub)].obj.values[0] in preds_arr:
        count_pred_dob += 1
  print(f'count: {count_pred_dob}, total cases number: {len(df)}, precision: {count_pred_dob/len(df)}')

count: 38, total cases number: 2034, precision: 0.018682399213372666
count: 63, total cases number: 2034, precision: 0.030973451327433628
count: 82, total cases number: 2034, precision: 0.04031465093411996
count: 104, total cases number: 2034, precision: 0.051130776794493606
count: 134, total cases number: 2034, precision: 0.06588003933136677


In [ ]:
k = 5
for test_k in range(k):
  count_pred_pob = 0
  for (sub, pred) in zip(df_2['subj'], df_2['pred']):
    if pred == '/people/person/place_of_birth':
      text = sub + " was born in [MASK]."
    else:
      print(f'{pred} is not a canonical case')
      break
    preds = mask_filler(text)
    if df_2[(df_2['subj']== sub)].obj.values[0] in [preds[selected_k]['token_str'] for selected_k in range(test_k+1)]:
      count_pred_pob += 1
  print(f'k: {test_k+1}, count: {count_pred_pob}, total cases number: {len(df_2)}, precision: {count_pred_pob/len(df_2)}')

k: 1, count: 352, total cases number: 3418, precision: 0.10298420128730251
k: 2, count: 480, total cases number: 3418, precision: 0.1404330017554125
k: 3, count: 568, total cases number: 3418, precision: 0.16617905207723815
k: 4, count: 651, total cases number: 3418, precision: 0.19046225863077823
k: 5, count: 716, total cases number: 3418, precision: 0.20947922761849033


### ***Place of death***

In [ ]:
df_3 = pd.read_json('/content/pod_augment-200526.json')
df_3 = df_3.where(df_3['maj_vote']=='yes')
df_3 = df_3.dropna(axis=0, how='all').reset_index()
for i in range(len(df_3.evidences)):
  df_3.evidences[i] = df_3.evidences[i][0].get('snippet')
df_3 = df_3[['pred', 'sub', 'obj', 'evidences']]
subj_new = []
obj_new = []
evidence_new = []
for subj, evid, obj in zip(df_3['sub'], df_3['evidences'], df_3['obj']):
  if obj in evid and len(tokenizer(evid)['input_ids']) < 100 and obj in vocab:
    subj_new.append(subj)
    obj_new.append(obj)
    evidence_new.append(evid)
pred_new = ['/people/person/place_of_death'] * len(subj_new)
df_3 = pd.DataFrame(list(zip(pred_new, subj_new, obj_new, evidence_new)), columns=df_3.columns.to_list())
df_3 = df_3.rename(columns={"sub": "subj"}, errors="raise")
df_3 = df_3.apply(lambda x: x.astype(str).str.lower())
df_3.obj = df_3.obj.apply(lambda x: unidecode.unidecode(x))
subj_new = []
obj_new = []
evidence_new = []
for subj,obj,evid in zip(df_3.subj,df_3.obj, df_3.evidences):
  if obj in evid:
    subj_new.append(subj)
    obj_new.append(obj)
    evidence_new.append(evid)

pred_new = ['/people/person/place_of_death'] * len(subj_new)
df_3 = pd.DataFrame(list(zip(pred_new, subj_new, obj_new, evidence_new)), columns=df_3.columns.to_list())
dataset_3 = Dataset.from_pandas(df_3)

<ipython-input-13-7b3f54ca341c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3.evidences[i] = df_3.evidences[i][0].get('snippet')


In [ ]:
count_pred_pod = 0
for (sub, pred) in zip(df_3['subj'], df_3['pred']):
  if pred == '/people/person/place_of_death':
    text = sub + " died in [MASK]."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  if df_3[(df_3['subj']== sub)].obj.values[0] == preds[0]['token_str']:
    count_pred_pod += 1
print(f'count: {count_pred_pod}, total cases number: {len(df_3)}, precision: {count_pred_pod/len(df_3)}')

count: 81, total cases number: 1188, precision: 0.06818181818181818


### ***Adding context***

In [ ]:
count_pred_pod = 0
for (sub, pred) in zip(df_3['subj'], df_3['pred']):
  if pred == '/people/person/place_of_death':
    text = "louis armstrong died in new york. " + sub + " died in [MASK]."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  if df_3[(df_3['subj']== sub)].obj.values[0] == preds[0]['token_str']:
    count_pred_pod += 1
print(f'count: {count_pred_pod}, total cases number: {len(df_3)}, precision: {count_pred_pod/len(df_3)}')

count: 67, total cases number: 1188, precision: 0.0563973063973064


In [ ]:
count_pred_pod = 0
for (sub, pred) in zip(df_3['subj'], df_3['pred']):
  if pred == '/people/person/place_of_death':
    text = "albert einstein died in princeton. louis armstrong died in new york. " + sub + " died in [MASK]."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  if df_3[(df_3['subj']== sub)].obj.values[0] == preds[0]['token_str']:
    count_pred_pod += 1
print(f'count: {count_pred_pod}, total cases number: {len(df_3)}, precision: {count_pred_pod/len(df_3)}')

count: 52, total cases number: 1188, precision: 0.04377104377104377


In [ ]:
count_pred_pod = 0
for (sub, pred) in zip(df_3['subj'], df_3['pred']):
  if pred == '/people/person/place_of_death':
    text = "luciano pavarotti died in modena. albert einstein died in princeton. louis armstrong died in new york. " + sub + " died in [MASK]."
  else:
    print(f'{pred} is not a canonical case')
    break
  preds = mask_filler(text)
  if df_3[(df_3['subj']== sub)].obj.values[0] == preds[0]['token_str']:
    count_pred_pod += 1
print(f'count: {count_pred_pod}, total cases number: {len(df_3)}, precision: {count_pred_pod/len(df_3)}')

count: 93, total cases number: 1188, precision: 0.07828282828282829
